In [1]:
!pip install catboost

     |████████████████████████████████| 67.4 MB 25 kB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from catboost import CatBoostClassifier
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [4]:
wrong_samples = pd.read_csv('drive/My Drive/wrong_samples_run.csv', index_col=0)

In [5]:
wrong_samples.columns = ['text', 'class_0', 'class']

In [6]:
wrong_samples['class'].value_counts()

7    43853
6     5714
Name: class, dtype: int64

In [7]:
idx = wrong_samples['class'] == 7
wrong_samples['class'][idx] = 0
wrong_samples['class'][~idx] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")


Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [10]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 55 kB 2.6 MB/s 
     |████████████████████████████████| 8.2 MB 12.5 MB/s 
     |████████████████████████████████| 371 kB 66.5 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=852279 sha256=5192705044b4eb6e7bda8d0a83df8ed74d5ad85e4ad23988b45d7bd22af0d9d9
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [11]:
from pymorphy2 import MorphAnalyzer
morph_analyzer = MorphAnalyzer()
stop_words = stopwords.words("russian")
import re

In [12]:
def preprocess(text):
  text = text.lower()
  text = re.sub('\s+', ' ', re.sub(r'[^А-Яа-яёЁ]', ' ', text).strip())
  text = [morph_analyzer.parse(word)[0].normal_form for word in text.split()]
  text = [word for word in text if word not in stop_words]
  if len(text) < 5:
    return None
  return ' '.join(text)

In [13]:
wrong_samples = wrong_samples[wrong_samples['text'].apply(type) == str]

In [14]:
wrong_samples['text'] = wrong_samples['text'].apply(preprocess)

In [15]:
wrong_samples = wrong_samples[wrong_samples['text'].apply(lambda x: x is not None)]

In [16]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=100)
X_tfidf = vectorizer.fit_transform(wrong_samples['text'].values.astype('U'))
X_tfidf = X_tfidf.toarray()

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, wrong_samples['class'], stratify=wrong_samples['class'], test_size=0.25)

In [19]:
grid = {
        'learning_rate': [0.01, 0.1, 0.5],
        'depth': [4, 6, 8],
        'l2_leaf_reg': [0.1, 0.5, 1],
        'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide'],
        'auto_class_weights': [None, 'Balanced', 'SqrtBalanced']
}

In [22]:
model = CatBoostClassifier(eval_metric='F1', iterations=100, verbose=0)
grid_search_result = model.grid_search(grid, 
                                       X=X_train, 
                                       y=y_train)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.


bestTest = 0.7075384048
bestIteration = 13

0:	loss: 0.7075384	best: 0.7075384 (0)	total: 2.02s	remaining: 8m 7s

bestTest = 0.6957772333
bestIteration = 3

1:	loss: 0.6957772	best: 0.7075384 (0)	total: 3.94s	remaining: 7m 55s

bestTest = 0.701842102
bestIteration = 3

2:	loss: 0.7018421	best: 0.7075384 (0)	total: 5.83s	remaining: 7m 46s

bestTest = 0.7144412866
bestIteration = 32

3:	loss: 0.7144413	best: 0.7144413 (3)	total: 7.72s	remaining: 7m 41s

bestTest = 0.6957772333
bestIteration = 3

4:	loss: 0.6957772	best: 0.7144413 (3)	total: 9.62s	remaining: 7m 37s

bestTest = 0.6980664341
bestIteration = 2

5:	loss: 0.6980664	best: 0.7144413 (3)	total: 11.5s	remaining: 7m 35s

bestTest = 0.7141532818
bestIteration = 32

6:	loss: 0.7141533	best: 0.7144413 (3)	total: 13.5s	remaining: 7m 33s

bestTest = 0.6957772333
bestIteration = 3

7:	loss: 0.6957772	best: 0.7144413 (3)	total: 15.4s	remaining: 7m 31s

bestTest = 0.6999611806
bestIteration = 3

8:	loss: 0.6999612	best: 0.7144413 (3)	tota

In [23]:
model = CatBoostClassifier(eval_metric='F1', iterations=100, verbose=0, **grid_search_result['params'])
model.fit(X_train, y_train, eval_set=(X_test, y_test))

In [25]:
model.save_model('color_model.bin')

In [27]:
import pickle
with open('tfidf_color.pickle', 'wb') as f:
  pickle.dump(vectorizer, f)